**Dans le dataset initial (5000_movies.csv)**, il y a beaucoup de valeurs manquantes dans `gross` et dans `budget` donc imputation délicate mais comme on veut garder ces variables, on va alors supprimer les lignes. Avant cette suppression on peut essayer de scraper IMDB pour obtenir des informations supplémentaires. Pour cela, un petit notebook séparé dans le même dossier permet de récupérer les données supplémentaires éventuelles, de les ajouter dans les colonnes `gross` et `budget` du dataframe `data`. Comme l'éxecution est un peu longue, on va sauvegarder ce dataframe dans un nouveau csv pour ne pas avoir à le refaire à chaque fois.

In [1]:
import numpy as np
import pandas as pd

from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
data = pd.read_csv('5000_movies.csv')

In [3]:
# scraping des données 'gross' et 'budget'
def scrap_gross_budget(row):
    link = row['movie_imdb_link']
    
    # si une des valeurs est manquante on va tenter d'aller les chercher
    if (np.isnan(row['gross'])) | (np.isnan(row['budget'])) :
        try :
            # on récupère la page web
            page = urlopen(link).read()
            soup = BeautifulSoup(page)

            # on récupère dans la page le Domestic Gross
            scrap_gross = soup.find('li', {'data-testid':'title-boxoffice-grossdomestic'}).find('span', {'class':"ipc-metadata-list-item__list-content-item"})
            scrap_gross = float(''.join(c for c in scrap_gross.contents[0] if c.isdigit()))

            # on récupère dans la page le Budget
            scrap_budget = soup.find('li', {'data-testid':'title-boxoffice-budget'}).find('span', {'class':"ipc-metadata-list-item__list-content-item"})
            scrap_budget = float(''.join(c for c in scrap_budget.contents[0] if c.isdigit()))
                    
        except AttributeError as err:
            return row[['gross', 'budget']]
    
    if (np.isnan(row['gross'])) & (np.isnan(row['budget'])) :
        return scrap_gross, scrap_budget
    
    elif (np.isnan(row['gross'])) :
        return scrap_gross, row['budget']
    
    elif (np.isnan(row['budget'])) :
        return row['gross'], scrap_budget
    
    else:
        return row[['gross', 'budget']]

In [4]:
from tqdm import tqdm
tqdm.pandas()
data[['gross', 'budget']] = data.progress_apply(scrap_gross_budget, axis=1)

100%|███████████████████████████████████████| 5043/5043 [54:46<00:00,  1.53it/s]


In [5]:
# enregistrement du dataframe en csv
data.to_csv('5000_movies_bis.csv', index=False)